In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing, load_digits, load_iris
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
import pandas as pd

import xgboost as xgb

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', None)


# df = pd.read_csv('entities-2023-09-02_17 33 41.csv', usecols=['start', 'mean'])
df = pd.read_csv('entities-2023-09-06_10 17 37.csv', usecols=['start', 'mean'])
df['start'] = pd.to_datetime(df['start'], format='%Y-%m-%d %H:%M:%S')
df = df.set_index('start')
df = df.asfreq(freq='H')
df = df[['mean']]
df = df.tail(10*7*24)
print(len(df) / 24)

# df['holiday'] = False
# df.loc['2023-08-25 00:00:00':'2023-09-03 00:00:00', 'holiday'] = True

df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek
df['is_weekend'] = (df.index.dayofweek >= 5)
df['is_weekend'] = df['is_weekend'].replace({True: 1, False: 0})
# df['day_so_far'] = df.groupby(df.index.day)['mean'].cumsum()
df['past_24h'] = df['mean'].rolling(24).sum().shift()
df['yesterday'] = df.groupby(df.index.date)['mean'].sum().shift(1) / 24
df['yesterday'] = df['yesterday'].fillna(method='ffill')
df['yesterday_2'] = df.groupby(df.index.date)['mean'].sum().shift(2) / 24
df['yesterday_2'] = df['yesterday_2'].fillna(method='ffill')
df['yesterday_peak'] = df.groupby(df.index.date)['mean'].max().shift(1)
df['yesterday_peak'] = df['yesterday_peak'].fillna(method='ffill')
df['hlag_1d'] = df['mean'].shift(24)
df['hlag_2d'] = df['mean'].shift(24*2)
df['hlag_3d'] = df['mean'].shift(24*3)
df['hlag_4d'] = df['mean'].shift(24*4)
df['hlag_5d'] = df['mean'].shift(24*5)
df['hlag_6d'] = df['mean'].shift(24*6)
df['hlag_7d'] = df['mean'].shift(24*7)
df['yesterday_25'] =  df.groupby(df.index.date)['mean'].quantile(0.8).shift(1) / 24
df['yesterday_25'] = df['yesterday_25'].fillna(method='ffill')

df['cum_sum_today'] = df.groupby(df.index.date)['mean'].cumsum().shift(1)
df['lag_1h'] = df['mean'].shift(1)
df['lag_2h'] = df['mean'].shift(2)
df['lag_3h'] = df['mean'].shift(3)

df['sum_today'] = df.groupby(df.index.date)['mean'].sum()
df['sum_today'] = df['sum_today'].fillna(method='ffill')
df['mean_today'] = df['sum_today'] / 24

# stl = """{"predicted":{"1689811200000":6.1024130505,"1689897600000":5.2882367582,"1689984000000":7.1046379757,"1690070400000":7.5558324369,"1690156800000":7.4207686308,"1690243200000":7.1760769877,"1690329600000":5.8848356135,"1690416000000":7.0620552752,"1690502400000":5.3280016617,"1690588800000":8.6262117886,"1690675200000":8.2011682559,"1690761600000":9.6541242131,"1690848000000":6.395082486,"1690934400000":6.2387428456,"1691020800000":6.0210112227,"1691107200000":5.9131599036,"1691193600000":6.8258694141,"1691280000000":6.067949583,"1691366400000":5.2549148829,"1691452800000":6.3373241805,"1691539200000":5.5160181919,"1691625600000":7.5489618675,"1691712000000":6.1272265017,"1691798400000":7.850609946,"1691884800000":8.1679921251,"1691971200000":8.8346228733,"1692057600000":7.1282213566,"1692144000000":6.6773378019,"1692230400000":6.6954823189,"1692316800000":7.0140371574,"1692403200000":7.5789723455,"1692489600000":6.9975712542,"1692576000000":7.1835653678,"1692662400000":6.4692300345,"1692748800000":6.2646061006,"1692835200000":6.5575620771,"1692921600000":7.4596137307,"1693008000000":6.8102031764,"1693094400000":6.0367357611,"1693180800000":4.9065749756,"1693267200000":4.1936456354,"1693353600000":4.334267945,"1693440000000":4.7481844442,"1693526400000":4.5254548537,"1693612800000":4.4046905524,"1693699200000":5.1102698367,"1693785600000":10.1143400729,"1693872000000":11.3499860392}}"""
# stl = pd.read_json(stl).rename(columns={"predicted": "stl_sum_today"}).asfreq('D')
# df = stl.combine_first(df)
# df['stl_sum_today'] = df['stl_sum_today'].fillna(method='ffill')
# df['stl_sum_today'] = df['stl_sum_today'].fillna(df['sum_today'])

exogs = ['hour', 'dayofweek', 'yesterday', 'yesterday_2', 'yesterday_peak','hlag_1d', 'hlag_2d', 'hlag_3d','hlag_4d','hlag_5d','hlag_6d', 'hlag_7d']
# exogs = ['hour', 'dayofweek', 'yesterday', 'yesterday_2', 'yesterday_peak', 'hlag_1d', 'hlag_2d', 'hlag_3d', 'hlag_7d']
# exogs = ['hour', 'dayofweek', 'yesterday', 'cum_sum_today', 'lag_1h', 'lag_2h', 'lag_3h']

# train_period = 24 * 7 * 9
train_period = 24*7*5
test_period = 24*7
# # for x in range(7*3):
for x in range(0, len(df) - train_period - test_period, 24):
    train_start = x
    train_end = x + train_period
    test_start = train_end
    test_end = test_start + test_period
    predict_start = test_end
    predict_end = predict_start + 24
    predict_start_date = df.iloc[predict_start].name

    print(f"Training from {train_start} - {train_end - 1}: {df.iloc[train_start].name} - {df.iloc[train_end - 1].name}")

    X_train = df[train_start:train_end][exogs]
    y_train = df[train_start:train_end]['mean']

    X_test = df[test_start:test_end][exogs]
    y_test = df[test_start:test_end]['mean']
    
    X_predict = df[predict_start:predict_end][exogs]

    params = {'max_depth': 6, 'min_child_weight': 2, 'eta': 0.3, 'subsample': 0.7, 'colsample_bytree': 0.7, 'objective': 'reg:squaredlogerror'}
    # model = xgb.XGBRegressor(verbosity=0, num_estimators=500, early_stopping_rounds=100, objective='reg:squaredlogerror')
    model = xgb.XGBRegressor(verbosity=0, **params)
    model.fit(X_train, y_train, verbose=False, eval_set=[(X_train, y_train), (X_test, y_test)])

    # xgb.plot_importance(model, height=0.9)

    prediction = model.predict(X_predict)
    forecasted_df = pd.DataFrame(prediction, columns=['predicted'], index=pd.date_range(predict_start_date, predict_start_date + pd.Timedelta(hours=23), freq='H'))
    df = df.combine_first(forecasted_df)




In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot

forecast_start = train_period + test_period

# df = df[['mean', 'predicted']]
print(df[forecast_start:][['mean', 'predicted']])

# df['predicted_sum'] = df.groupby(df.index.date)['predicted'].sum()
# df['predicted_sum'] = df['predicted_sum'].fillna(method='ffill')

# df['predicted_norm'] = df['predicted'] * df['stl_sum_today'] / df['predicted_sum']



daily_sums = df[forecast_start:].resample('D').sum()
# daily_sums['stl_sum_today'] = daily_sums['stl_sum_today'] / 24

pyplot.figure()
fig, ax = pyplot.subplots(figsize=(15, 5))
df[forecast_start-48:]['mean'].plot(ax=ax)
df[forecast_start-48:]['predicted'].plot(ax=ax)
# df[forecast_start-48:]['predicted_norm'].plot(ax=ax)
# (daily_sums['mean'] - daily_sums['predicted']).shift(1).plot(ax=ax)
# (df[forecast_start-48:]['mean'] - df[forecast_start-48:]['predicted']).plot(ax=ax)
# ax.fill_between(df.index, df['predicted_lower'], df['predicted_upper'], color='k', alpha=0.1);  
pyplot.legend()
pyplot.show()

print(mean_absolute_error(df[forecast_start:]['mean'], df[forecast_start:]['predicted']))

pyplot.figure()
fig, ax = pyplot.subplots(figsize=(15, 5))
daily_sums['mean'].plot(ax=ax)
daily_sums['predicted'].plot(ax=ax)
# daily_sums['predicted_norm'].plot(ax=ax)
# daily_sums['stl_sum_today'].plot(ax=ax)
pyplot.legend()
pyplot.show()


# print(mean_absolute_error(df[forecast_start:]['mean'], df[forecast_start:]['predicted_norm']))
print(mean_absolute_error(df[forecast_start:]['mean'], df[forecast_start:]['predicted']))
# print(daily_sums[['mean', 'predicted', 'predicted_norm']])
print(daily_sums[['mean', 'predicted']])
# print(mean_absolute_error(daily_sums['mean'], daily_sums['predicted_norm']))
# print(mean_squared_error(daily_sums['mean'], daily_sums['predicted_norm']))
print(mean_absolute_error(daily_sums['mean'], daily_sums['predicted']))
print(mean_squared_error(daily_sums['mean'], daily_sums['predicted']))

In [ ]:
# https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

train_period = 7*10
test_period = 7*2

train_start = 0
train_end = train_period

X_train = df[train_start:train_end][exogs]
y_train = df[train_start:train_end]['mean']

# X_test = df[test_start:test_end][exogs]
# y_test = df[test_start:test_end]['mean']

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    # Parameters that we are going to tune.
    'max_depth':8,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squaredlogerror',
}
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(1,10)
    for min_child_weight in range(1,10)
]
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=500,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=100
    )    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [ ]:
params['max_depth'] = 6
params['min_child_weight'] = 2

gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]
min_mae = float("Inf")
best_params = None# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=500,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=100
    )    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

In [ ]:
params['subsample'] = 0.7
params['colsample_bytree'] = 0.7

min_mae = float("Inf")
best_params = None

for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))    # We update our parameters
    params['eta'] = eta    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=100,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
          )    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

In [ ]:
params['eta'] = .3

print(params)

In [ ]:
from statistics import LinearRegression
from mlforecast import MLForecast
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from window_ops.rolling import rolling_mean, rolling_max, rolling_min

df['unique_id'] = 'id'
df['ds'] = df.index
# df = df.drop(columns=['past_24h'])
df['past_24h'] = df['mean'].rolling(24).sum().shift()
input = df.copy()

for x in range(0, len(df) - train_period, 24):
# for x in range(1):
    train_start = x
    train_end = x + train_period
    predict_start = input.iloc[train_end].name

    print(f"Training from {train_start} - {train_end - 1}: {input.iloc[train_start].name} - {input.iloc[train_end - 1].name}")

    # X_train = input[train_start:train_end][exogs]
    y_train = input[train_start:train_end]['mean']

    # X_test = input[train_end:train_end+24][exogs]
    y_test = input[train_end:train_end+24]['mean']

    models = [xgb.XGBRegressor(), HistGradientBoostingRegressor()]

    model = MLForecast(models=models,
                    freq='H',
                    lags=[24,24*7],
                    lag_transforms={
                        2: [
                            (rolling_mean, 24),
                            (rolling_mean, 24*7),
                        ]
                    },
                    date_features=['hour', 'dayofweek'],
                    
                    )

    model.fit(input[train_start:train_end], id_col='unique_id', time_col='ds', target_col='mean', static_features=[])
    print(input[train_end:train_end+24][['past_24h']])
    pred = model.predict(horizon=24, dynamic_dfs=input[train_end:train_end+24][['past_24h']])
    pred.index = pred['ds']
    pred = pred[['XGBRegressor']]
    pred = pred.rename(columns={'XGBRegressor': 'predicted'})

    # model = xgb.XGBRegressor(verbosity=0, objective="reg:absoluteerror", eval_metric="mae")
    # model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=False)

    # xgb.plot_importance(model, height=0.9)

    # prediction = model.predict(X_test)
    # forecasted_df = pd.DataFrame(prediction, columns=['predicted'], index=pd.date_range(predict_start, predict_start + pd.Timedelta(hours=23), freq='H'))
    df = df.combine_first(pred)



In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot

df = df[['mean', 'predicted']]

pyplot.figure()
fig, ax = pyplot.subplots(figsize=(15, 5))
df[train_period-48:]['mean'].plot(ax=ax)
df[train_period-48:]['predicted'].plot(ax=ax)
# (df[train_period-48:]['mean'] - df[train_period-48:]['predicted']).plot(ax=ax)
# ax.fill_between(df.index, df['predicted_lower'], df['predicted_upper'], color='k', alpha=0.1);  
pyplot.legend()
pyplot.show()

daily_sums = df[train_period:].resample('D').sum()
print(mean_absolute_error(df[train_period:]['mean'], df[train_period:]['predicted']))
print(daily_sums)
print(mean_squared_error(daily_sums['mean'], daily_sums['predicted']))
print(mean_absolute_error(daily_sums['mean'], daily_sums['predicted']))